# Final Project: Rijksmuseum Multi-label Classification Tasks
## CSC 594: Advanced Deep Learning
## Ross Pollock

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras 
import tensorflow.keras.layers as L

from tensorflow_addons.losses import SigmoidFocalCrossEntropy

import numpy as np 
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import average_precision_score
from functools import reduce
from collections import  Counter
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
# Extract Data to local colab drive
! tar xfz /drive/My\ Drive/CSC594/Data/Image/Rijksmuseum.tar.gz -C /content/

In [ ]:


'''
Implementation of ResNet 50,101,152
with optional width parameter for 
making wider ResNets
'''

class ResNetBlock(L.Layer):
  def __init__(
      self,n_filters,regularizer=keras.regularizers.L2(1e-3),
      activation=L.ReLU,start=False,downsample=False
    ):
    super(ResNetBlock,self).__init__()
    self.regularizer = regularizer
    self.activation = activation
    self.n_filters = n_filters
    self.start = start
    self.stride = 2 if downsample else 1


    self.left_path = keras.Sequential(
        [
          L.Conv2D(self.n_filters,1,self.stride,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization(),
          self.activation(),
          L.Conv2D(self.n_filters,3,1,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization(),
          self.activation(),
          L.Conv2D(self.n_filters*4,1,1,padding='same',kernel_regularizer=self.regularizer),
          L.BatchNormalization()
        ]
    )
  
    # If first block add convolution to residual path
    # so that the number of output channels match
    if self.start:
      self.right_path = keras.Sequential(
          [
           L.Conv2D(self.n_filters*4,1,self.stride,'same',kernel_regularizer=self.regularizer),
           L.BatchNormalization()
          ]
      )

    

  def call(self,input_tensor,training=False):
    x = self.left_path(input_tensor)
    if self.start:
      y = self.right_path(input_tensor)
    else:
      y = input_tensor

    z = tf.add(x,y)
    return self.activation()(z)
      

class ResNetStack(L.Layer):
  def __init__(self,
      n_blocks,n_filters,
      first_block=False,**kwargs
    ):
    super(ResNetStack,self).__init__()
    blocks = [ 
      ResNetBlock(n_filters,start=(not i),downsample=(not i and not first_block),**kwargs) 
      for i in range(n_blocks)
    ]
    self.stack = keras.Sequential(
        blocks
    )
    self.out_dim = n_filters * 4

  def call(self,input_tensor,training=False):
    return self.stack(input_tensor) 

def ResNet(n_layers,width=1,input_shape=None,**kwargs):
    if n_layers not in [50,101,152]:
      raise ValueError
    
    model = keras.Sequential([
        L.Input(input_shape),
        L.Conv2D(64,7,2,padding='same'),
        L.BatchNormalization(),
        L.ReLU(),
        L.MaxPool2D(3,2,'same')
        
    ],name=f'ResNet{n_layers}')


    model_specs = {
        50: [3,4,6,3],
        101: [3,4,23,3],
        152: [3,8,36,3]
    }
    filters = list(map(lambda x: x*width,[64,128,256,512]))

    for i,(stack,f) in enumerate(zip(model_specs[n_layers],filters)):
      if i == 0:
        stack = ResNetStack(stack,f,first_block=True)
      else:
        stack = ResNetStack(stack,f,first_block=False)
      model.add(stack)
    model.add(L.GlobalAveragePooling2D())
    return model




## Load Pretrained Model

In [ ]:
model_url = 'https://rosspollock.design/models/resnet50x2_120k.tar.gz'
keras.utils.get_file('modelx2120',model_url,untar=True,extract=True)
encoder = keras.models.load_model('/root/.keras/datasets/query_encoder120k')
encoder = encoder.get_layer('ResNet50')
rep_dim = encoder.output_shape[1]

In [ ]:
if encoder.name == 'ResNet50':
  new_encoder = ResNet(50,2,input_shape=(244,244,3))
  new_encoder.set_weights(encoder.get_weights())
  encoder = new_encoder 
  del new_encoder

In [ ]:
@tf.function
def smart_resize(img,size=244):
  h,w = tf.shape(img)[0], tf.shape(img)[1]
  min_dim = tf.minimum(h,w)
  ratio = size / tf.cast(min_dim,tf.float32) 

  new_w = tf.cast(w,tf.float32) * ratio
  new_h = tf.cast(h,tf.float32) * ratio

  img = tf.image.resize(img,[new_h,new_w],preserve_aspect_ratio=True)
  img = tf.image.resize_with_crop_or_pad(img,size,size)
  return img

@tf.function
def img_preprocess(fpath):
    img = tf.io.read_file(fpath)
    img = tf.image.decode_jpeg(img,3)
    img = tf.cast(img,tf.float32)
    img = smart_resize(img,size=244)
    img /= (255/2)
    img -= 1 
    return img 

## Load Metadata Files 
### Get Image Paths

In [ ]:
test_df = pd.read_csv('/drive/My Drive/CSC594/Data/Rijkmuseum_test_metadata.csv')
train_df = pd.read_csv('/drive/My Drive/CSC594/Data/Rijkmuseum_train_metadata.csv')



In [ ]:
train_paths = train_df.object_path.map(lambda x: '/content/smaller/'+x).to_list()
test_paths = test_df.object_path.map(lambda x: '/content/smaller/'+x).to_list()

## Material Classification

In [ ]:
train_materials = train_df['object_materials'].map(lambda x: str(x).split('|')).to_list()
test_materials = test_df['object_materials'].map(lambda x: str(x).split('|')).to_list()

In [ ]:
mat_counter = Counter()
for mat_list in test_df['object_materials'].map(lambda x: str(x).split('|')).to_list():
  mat_counter.update(mat_list)

In [ ]:
material_classes = []
for k, v in mat_counter.items():
  if k == '': continue
  if v > 10:
    material_classes.append(k)

In [ ]:
Material_Biner = MultiLabelBinarizer(material_classes)
train_materials = Material_Biner.fit_transform(train_materials)
test_materials = Material_Biner.transform(test_materials)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['', 'Berlijns zilver', 'Chinees papier', 'Delfts aardewerk', 'Engels aardewerk', 'Oost-Indische inkt', 'acaciahout', 'acajouhout', 'agaat', 'albast', 'aluminium', 'amalgaam', 'amboinahout', 'amethist', 'andesiet', 'atlas liseré', 'aventurienglas', 'baai (materiaal)', 'balein', 'bamboe', 'bandagaat', 'barnsteen', 'barokparel', 'been', 'beenderporselein', 'beits', 'berberishout', 'bergkristal', 'berkenhout', 'biscuit', 'bister', 'blaas', 'bladgoud', 'bladzilver', 'blanc de Chine', 'bloedkoraal', 'bolletriehout', 'bolus', 'bombazijn', 'bont', 'bordpapier', 'brandstof', 'bristolkarton', 'brokaat', 'bronspoeder', 'bruinharthout', 'buffelhoorn', 'buffelleer', 'buxushout', 'calqueerpapier', 'cantilledraad', 'carneool', 'cederhout', 'celadon', 'celluloid', 'cellulose acetaat', 'chalcedoon', 'chelas', 'chenille', 'chrysopraas', 'citrien', 'citroenboomhout', 'coromandelhout', 'coromandella

In [ ]:
Material_Biner.classes_.shape[0]

78

In [ ]:
n_train = train_materials.shape[0]
n_test = test_materials.shape[0]

In [ ]:
x_train = np.zeros((n_train,rep_dim))
x_test = np.zeros((n_test,rep_dim))

In [ ]:
BATCH_SIZE = 128
ds_train = tf.data.Dataset.from_tensor_slices(train_paths)
ds_train = ds_train.map(img_preprocess)
ds_train = ds_train.batch(BATCH_SIZE)

ds_test = tf.data.Dataset.from_tensor_slices(test_paths)
ds_test = ds_test.map(img_preprocess)
ds_test = ds_test.batch(BATCH_SIZE)

In [ ]:
pbar = keras.utils.Progbar(n_train // BATCH_SIZE + 1 )
for i, batch in enumerate(ds_train):
  reps = encoder(batch,training=False)
  x_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = reps.numpy()
  pbar.update(i+1)

pbar = keras.utils.Progbar(n_test // BATCH_SIZE + 1 )
for i, batch in enumerate(ds_test):
  reps = encoder(batch,training=False)
  x_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = reps.numpy()
  pbar.update(i+1)

176/176 [==============================] - 127s 721ms/step


In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
clf = keras.Sequential([
  L.Dense(Material_Biner.classes_.shape[0],'sigmoid')
])


In [ ]:
clf.compile(
    keras.optimizers.Adam(),
    SigmoidFocalCrossEntropy()
)

In [ ]:
train_materials = train_materials.astype(np.float32)

In [ ]:
clf.fit(x=x_train,y=train_materials,epochs=50,validation_split=0.2)

Epoch 1/50
2241/2241 [==============================] - 6s 3ms/step - loss: 3.0748 - val_loss: 0.7612
Epoch 2/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.4230 - val_loss: 0.2423
Epoch 3/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1844 - val_loss: 0.1468
Epoch 4/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1326 - val_loss: 0.1233
Epoch 5/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1178 - val_loss: 0.1167
Epoch 6/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1124 - val_loss: 0.1142
Epoch 7/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1095 - val_loss: 0.1119
Epoch 8/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1075 - val_loss: 0.1142
Epoch 9/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1067 - val_loss: 0.1112
Epoch 10/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.106

In [ ]:
test_pred = clf.predict(x_test)

In [ ]:
iMAP = average_precision_score(
    test_materials[~np.all(test_materials==0,1),:],
    test_pred[~np.all(test_materials==0,1),:],
    average='samples')

In [ ]:
print(f'{test_materials.shape[1]} Material Multi-Label Classification iMAP: {iMAP:0.4f}')

78 Material Multi-Label Classification iMAP: 0.9344


In [ ]:
MAP = average_precision_score(
    test_materials,
    test_pred,
    average='macro'
)
print(f'{test_materials.shape[1]} Material Multi-Label Classification MAP: {MAP:0.4f}')

78 Material Multi-Label Classification MAP: 0.2903


## Type Classification

In [ ]:
train_types = train_df['object_types'].map(lambda x: str(x).split('|')).to_list()
test_types = test_df['object_types'].map(lambda x: str(x).split('|')).to_list()

In [ ]:
type_counter = Counter()
for type_list in test_df['object_types'].map(lambda x: str(x).split('|')).to_list():
  type_counter.update(type_list)

type_classes = []
for k, v in type_counter.items():
  if k == '': continue
  if v > 10:
    type_classes.append(k)



In [ ]:
Type_Biner = MultiLabelBinarizer(type_classes)
train_types = Type_Biner.fit_transform(train_types).astype(np.float32)
test_types = Type_Biner.transform(test_types)
n_types = Type_Biner.classes_.shape[0]
print(f'Number of Art Types: {n_types}')

Number of Art Types: 101


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Avalokitesvara', 'Berlijns porselein', 'Boeddha-beeld', 'Brahma', 'Chinees porselein', 'Delftse faïence', 'Duits porselein', 'Europees porselein', 'Fez', 'Frankenthal porselein', 'Fulda porselein', 'Fürstenberg porselein', 'Hansje-in-de-kelder', 'Höchst porselein', 'Jacobakan', 'Johannesschotel', 'Krishna', 'Kuvera', 'Lokeshvara', 'Mahishasura', 'Maitreya', 'Manjushri', 'Nandin', 'Nandishvara', 'Perzische miniatuur', 'Rabat', 'Savonnerie-tapijt', 'Shiva', 'Sinclairsabel', 'Thang-ka', 'Thérèse', 'Visjnoe', 'Yamantaka', 'aak', 'aanplakbiljet', 'aardappelmaat', 'aardewerk', 'achterglasschildering', 'achterplaat', 'adreskaart', 'adverteermiddel', 'affuit', 'afgietsel', 'aflaatprent', 'agraaf', 'aker (versiersel)', 'albarello', 'albestrook', 'almanak', 'altaar', 'altaarhanger', 'altaarkruis', 'altaarlessenaar', 'amazonekostuum', 'ambtsketen', 'ameublement', 'ampul', 'anker (scheepsui

In [ ]:
type_clf = keras.Sequential([
  L.Dense(n_types,'sigmoid')
])

type_clf.compile(
    keras.optimizers.Adam(),
    SigmoidFocalCrossEntropy()
)

type_clf.fit(x=x_train,y=train_types,epochs=50,validation_split=0.2)

Epoch 1/50
2241/2241 [==============================] - 6s 3ms/step - loss: 3.9974 - val_loss: 1.0118
Epoch 2/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.5730 - val_loss: 0.3416
Epoch 3/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.2632 - val_loss: 0.2182
Epoch 4/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1954 - val_loss: 0.1837
Epoch 5/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1752 - val_loss: 0.1759
Epoch 6/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1677 - val_loss: 0.1715
Epoch 7/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1644 - val_loss: 0.1685
Epoch 8/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1622 - val_loss: 0.1677
Epoch 9/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.1606 - val_loss: 0.1672
Epoch 10/50
2241/2241 [==============================] - 6s 3ms/step - loss: 0.159

In [ ]:
test_pred = type_clf.predict(x_test)

iMAP = average_precision_score(
    test_types[~np.all(test_types==0,1),:],
    test_pred[~np.all(test_types==0,1),:],
    average='samples')

print(f'{n_types} Art Type Multi-Label Classification iMAP: {iMAP:0.4f}')

MAP = average_precision_score(
    test_types,
    test_pred,
    average='macro'
)
print(f'{n_types} Art Type Multi-Label Classification MAP: {MAP:0.4f}')

101 Art Type Multi-Label Classification iMAP: 0.8898
101 Art Type Multi-Label Classification MAP: 0.3875
